In [ ]:
!git clone https://github.com/FedericoArcelaschi-Polimi/edited_RecSyS_Course_AT_PoliMi

In [ ]:
cd edited_RecSyS_Course_AT_PoliMi

In [ ]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
import optuna
import pickle
%matplotlib inline
#%load_ext Cython
from sqlalchemy import create_engine

In [ ]:
#!python run_compile_all_cython.py
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

In [ ]:
!pip install pymysql

In [ ]:
data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
# data_train_path="data_train.csv"
# data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [ ]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
# user_map = {i : user for i, user in enumerate(data_target["user_id"])}
# item_map_inv = {item : i for i, item in item_map.items()}
# user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

In [ ]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
def train_evaluate(optuna_trial):
    factors = optuna_trial.suggest_int("factors", 1, 1000)
    alpha = optuna_trial.suggest_float("alpha", 0.0, 1.0)
    epsilon = optuna_trial.suggest_float("epsilon", 0.0, 1.0)
    reg = optuna_trial.suggest_float("reg", 0.0, 0.1)
    init_mean = optuna_trial.suggest_float("init_mean", -1.0, 1.0)
    init_std= optuna_trial.suggest_float("init_std", 0.0, 1.0)

    mAP = 0.0
    for i in range(0, test_folds):
        IALS_recommender = IALSRecommender(URM_trains[i], verbose=False)
        IALS_recommender.fit(num_factors=factors, reg=reg, alpha=alpha, epsilon=epsilon, init_mean=init_mean, init_std=init_std)
        result_df, _ = evaluator_tests[i].evaluateRecommender(IALS_recommender)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [ ]:
# Replace the following with your MySQL database connection details
host = "db-mysql-recsys-do-user-15390069-0.c.db.ondigitalocean.com"
port = 25060
database_name = "optuna"
username = "doadmin"
password = "AVNS_5Br3hCFa4U-D__KSaCl"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
mysql_url

study_name = "MF_IALS_DB"
study = optuna.load_study(study_name=study_name, storage=mysql_url)
study.trials_dataframe()

In [ ]:
study.optimize(train_evaluate, n_trials=50)


In [ ]:
study_df = study.trials_dataframe()
study_df.to_csv('../study_results.csv', index=False)
study_df

In [ ]:
import time
factors = study.best_trial.params["factors"]
alpha = study.best_trial.params["alpha"]
epsilon = study.best_trial.params["epsilon"]
reg = study.best_trial.params["reg"]
init_mean = study.best_trial.params["init_mean"]
init_std= study.best_trial.params["init_std"]

print(f"MAP : {study.best_trial.value}")
print(f"Hyperparameters: factors = {factors}, learning_rate = {learning_rate}")

IALS_recommender = IALSRecommender(URM_trains[i], verbose=False)
IALS_recommender.fit(num_factors=factors, reg=reg, alpha=alpha, epsilon=epsilon, init_mean=init_mean, init_std=init_std)
n_users_to_test = len(data_target["user_id"])
suggestions = pd.DataFrame(columns = ["user_id", "item_list"])
start_time = time.time()

for user_id in (data_target["user_id"]-1):
    suggestion = IALS_recommender.recommend(user_id, cutoff=10)
    suggestions.loc[len(suggestions)] = [user_id+1, " ".join([str(item_map[x]) for x in suggestion])] #" ".join([str(x) for x in np.flip(np.sort(suggestion_values))[0][:10]]
    
end_time = time.time()

print("Reasonable implementation speed is {:.2f} usr/sec".format(n_users_to_test/(end_time-start_time)))
suggestions.to_csv('../MF_MSE.csv', index = False)